## Estructura de los Datos

### Cash_Request (CR)

* **CR.id**:
  Unique ID of Cash Request.

* **CR.user_id**:
  Unique ID of the user who requested the cash advance.

* **CR.deleted_account_id**:
  If a user delete his account, we are replacing the user_id by this id. 
  
  It corresponds to a unique ID in the deleted account table with some keys information saved for fraud-fighting purposes (while respecting GDPR regulation).

* **CR.amount**:
  Amount of the Cash Request.

* **CR.created_at**:
  Timestamp of the CR creation.

* **CR.updated_at**:
  Timestamp of the latest CR's details update **(= update of at least one column in this table)**.

* **CR.reimbursement_date**:
  Planned reimbursement date. The user card will be charged at this date.

* **CR.cash_request_received_date**:
  Date of the receipt of the CR. Based on user's bank history.

* **CR.money_back_date**:
  Date where the CR was considered as money back. 
  
  It's either the paid_by_card date or the date were we considered that's the direc debit have low odds to be rejected (based on business rules) 

* **CR.send_at**:
  Timestamp of the funds's transfer.

* **CR.transfer_type**:
  Possible values are:

  | Campo        | Registros | Descripcion                                                  |
  | ------------ | --------: | ------------------------------------------------------------ |
  | **instant**: |     13882 | El usuario eligió recibir el adelanto instantáneamente. (user choose not received the advance instantly) |
  | **regular**: |     10086 | El usuario eligió no pagar inmediatamente y esperar la transferencia. (user choose to not pay and wait for the transfer) |

---

* **CR.reco_creation**:
  Timestamp of the recovery creation.
* **CR.reco_last_update**:
  Timestamp of the last recovery case update. Can be used to determine the incident closure date.
* **CR.moderated_at**:
  Timestamp of the manual review. Only filled if the CR needed a manual review.

* **CR.recovery_status**:
  Possible values are:

  | Campo                     | Registros | Descripcion                                                  |
  | ------------------------- | --------: | ------------------------------------------------------------ |
  | **NULL (Nice)**:          |     20639 | El CR nunca tuvo un incidente de pago. (Null if the cash request never had a payment incident.) |
  | **completed**:            |      2467 | El incidente de pago fue resuelto (el CR fue reembolsado). (the payment incident was resolved **(=the cash request was reimbursed)**) |
  | **pending**: (+-=direct_debit_rejected): |       845 | El incidente de pago aún está abierto. (the payment incident still open) |
  | **pending_direct_debit**: |        16 | El incidente de pago sigue abierto, pero se ha lanzado un débito directo SEPA. (the payment incident still open but a SEPA direct debit is launched) |
  | **cancelled**:            |         1 | ?? No figura a la documentacio                               |


* **CR.status**:     (23970 registros)
  Status of the CR. Possible values are: 

  | Campo                                                     | Registros | Descripcion                                                  |
  | --------------------------------------------------------- | --------: | ------------------------------------------------------------ |
  | **money_back**:                                           |     16395 | El CR fue reembolsado exitosamente.(The CR was successfully reimbursed.) |
  | **active**:                                               |        59 | Los fondos fueron recibidos en la cuenta del cliente. (Funds were received on the customer account.) |
  | **direct_debit_sent**:                                    |        34 | Se envió un débito directo SEPA, pero aún no se confirma el resultado. (We sent/scheduled a SEPA direct debit to charge the customer account. The result of this debit is not yet confirmed) |
  |                                                           |           |                                                              |
  | **rejected**:                                             |      6568 | El CR necesitó una revisión manual y fue rechazado. (The CR needed a manual review and was rejected) |
  | **direct_debit_rejected**: (+-=CR.recovery_status)        |       831 | El intento de débito directo SEPA falló. (Our last attempt of SEPA direct debit to charge the customer was rejected ) |
  | **transaction_declined**:                                 |        48 | No se pudo enviar el dinero al cliente. (We failed to send the funds to the customer) |
  | **canceled**:                                             |        33 | El usuario no confirmó el CR en la app, fue cancelado automáticamente. (The user didn't confirm the cash request in-app, we automatically canceled it) |
  | **En los datos proporcionados, NO aparecen los valores:** |           |                                                              |
  | **approved**:                                             |         0 | CR is a 'regular' one (= without fees) and was approved either automatically or manually. Funds will be sent aprox. 7 days after the creation. |
  | **money_sent**:                                           |         0 | We transferred the fund to the customer account. Will change to active once we detect that the user received the funds (using user's bank history). |
  | **pending**:                                              |         0 | The CR is pending a manual review from an analyst.           |
  | **waiting_user_confirmation**:                            |         0 | The user needs to confirm in-app that he want the CR (for legal reasons) |
  | **waiting_reimbursement**:                                |         0 | We were not able to estimate a date of reimbursement, the user needs to choose one in the app. |
     

---
---
#### Columnas que figuran en la documentacion per que NO TENEMOS EN LA BASE DE DATOS:

* **(CR.reason)**:
  Filled only if the CR was manually reviewed and rejected. **That's the rejection's reason displayed in-app**. 

* **(CR.cash_request_debited_date)**:
  Filled only if a SEPA direct debit was sent. It's the date were the latest direct debit was seen on the user account.


### Fees (FE)

* **FE.id**:
  Unique ID of the fee object.

* **FE.cash_request_id**:
  Unique ID of the CR linked to this fee.

* **FE.total_amount**:
  Amount of the fee (including VAT). **TODO: Es un importe o un porcentaje ??**

* **FE.reason**:
  Description of the fee.

* **FE.created_at**:
  Timestamp of the fee's creation.

* **FE.updated_at**:
  Timestamp of the latest fee's details update.

* **FE.paid_at**:
  Timestamp of the fee's payment.

* **FE.from_date**:
  Apply only to postpone fees. Initial date of reimbursement for the CR.

* **FE.to_date**:
  Apply only to postpone fees. New date of reimbursement for the CR.


* **FE.type**:
  Type of fee. Possible values are:

  | Campo                | Registros | Descripcion                                                  |
  | -------------------- | --------: | ------------------------------------------------------------ |
  | **instant_payment**: |     11095 | Fees por adelanto instantáneo. (fees for instant cash request (send directly after user's request, **through SEPA Instant Payment**) ) |
  | **postpone**:        |      7766 | Fees por la solicitud de posponer un reembolso. (fees created when a user want to postpone the reimbursment of a CR) |
  | **incident**:        |      2196 | Fees por fallos de reembolsos. (fees for failed reimbursement. Created after a failed direct debit) |
  | **split_payment**:   |         0 | Fees por pago fraccionado (en caso de un incidente).<br/>(futures fees for split payment (in case of an incident, we'll soon offer the possibility to our users to reimburse in multiples installements)) |


* **FE.status**:
  (= does the fees was successfully charged). Possibles values are:

  | Campo          | Registros | Descripcion                                                  |
  | -------------- | --------: | ------------------------------------------------------------ |
  | **accepted**:  |     14841 | El fee fue cobrado exitosamente. (fees were successfully charged) |
  | **cancelled**: |      4934 | El fee fue **creado pero cancelado** por algún motivo. <br>Se utiliza para solucionar problemas con las `fees`, pero se refiere principalmente a `fees` `postpone` que fallaron. Estamos cobrando las `fees` en el momento de la solicitud del `postpone`. <br/ >Si falla, no se acepta el `postpone` y la fecha de reembolso sigue siendo la misma. |
  |                |           | (Fee was created and cancelled for some reasons. It's used to fix issues with fees but it mainly concern postpone fees who failed. We are charging the fees at the moment of the postpone request. If it failed, the postpone is not accepted and the reimbursement date still the same.) |
  | **rejected**:  |      1194 | El último intento de cobrar el fee falló. (the last attempt to charge the fee failed.) |
  | **confirmed**: |        88 | El usuario completó una acción que creó un fee. (the user made an action who created a fee. <br/>It will normally get charged at the moment of the CR's reimbursement. In some rare cases, postpones are confirmed without being charges due to a commercial offer.) |


* **FE.category**:
  Describe the reason of the incident fee. 2 possibles values:

  | Campo                       | Registros | Descripcion                                                  |
  | --------------------------- | --------: | ------------------------------------------------------------ |
  | **rejected_direct_debit**:  |     18861 | Fees creados cuando el banco del usuario rechaza el primer débito directo. (fees created when user's bank rejects the first direct debit) |
  | **month_delay_on_payment**: |      1599 | Fees creados cada mes hasta que el incidente se cierre. (fees created every month until the incident is closed) |
  | **NULL**:                   |       597 | ?? No figura a la documentacio                               |


* **FE.charge_moment**:
  When the fee will be charge. 2 possibles values:

  | Campo       | Registros | Descripcion                                                  |
  | ----------- | --------: | ------------------------------------------------------------ |
  | **after**:  |     16720 | El fee se cobra cuando el CR es reembolsado. (the fee should be charged at the moment of the CR's reimbursement) |
  | **before**: |      4337 | El fee se cobra en el momento de su creación. (the fee should be charged at the moment of its creation) |

## EDA

In [82]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm

cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')

#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = pm.sort("df_jo", ['created_at','created_at_fe']).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()
df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()

### Estudio de registros

In [ ]:
counts = cr_cp.status.value_counts()
display(counts)

counts = cr_cp.transfer_type.value_counts()
display(counts)

counts = cr_cp.recovery_status.value_counts()
display(counts)

In [ ]:
counts = fe_cp.status.value_counts()
display(counts)

counts = fe_cp.type.value_counts()
display(counts)

counts = fe_cp.category.value_counts()
display(counts)

counts = fe_cp.charge_moment.value_counts()
display(counts)


In [85]:
good_cr = ['approved', 'money_sent', 'pending', 'direct_debit_sent', 'active', 'money_back']
good_fe = ['confirmed', 'accepted', 'cr_regular']
df_jo['needs_m_check'] = ~((df_jo['stat_cr'].isin(good_cr)) & (df_jo['stat_fe'].isin(good_fe))).astype(int)
display(df_jo.head(40))

,id_cr,id_fe,fe_cr_id,user_id,active,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status,updated_at,reco_creation,reco_last_update,updated_at_fe,Mes_created_at,cash_request_received_date,n_fees,n_backs,created_at_slot,created_at_slot_h,created_at_dow,needs_m_check
0,3,0,NaN,47,1,2019-11-19 13:57:53.511561,NaT,1.0,NaN,canceled,NaN,regular,nice,NaT,NaT,16 days 09:02:06.488439,NaT,NaT,NaT,NaT,NaT,NaT,2019-12-05 23:00:00.000000,NaT,NaN,2019-11-20 12:16:50.000000,NaN,NaN,NaT,NaT,nice,2020-12-14 10:25:57.710844,NaT,NaT,NaT,2019-11,NaT,0,0,7,13-Mañana,1,-1
1,4,0,NaN,99001309,0,2019-12-09 14:47:35.190714,NaT,100.0,NaN,money_back,NaN,regular,nice,NaT,NaT,7 days 08:12:24.809286,NaT,0 days 00:00:00,NaT,NaT,NaT,2019-12-16 23:00:00.000000,2019-12-16 23:00:00.000000,NaT,NaN,2019-12-09 14:58:28.000000,NaN,NaN,NaT,NaT,nice,2020-11-04 12:54:52.120014,NaT,NaT,NaT,2019-12,NaT,0,1,14,14-Tarde,0,-1
2,5,0,NaN,804,1,2019-12-10 19:05:21.596873,NaT,100.0,NaN,rejected,NaN,regular,nice,NaT,NaT,29 days 23:59:59.999490,NaT,NaT,NaT,NaT,NaT,NaT,2020-01-09 19:05:21.596363,NaT,NaN,2019-12-11 16:47:42.405646,NaN,NaN,NaT,NaT,nice,2019-12-11 16:47:42.407830,NaT,NaT,NaT,2019-12,NaT,0,0,14,19-Tarde,1,-1
3,6,0,NaN,812,1,2019-12-10 19:05:48.921042,NaT,100.0,NaN,direct_debit_rejected,NaN,regular,nice,0 days 04:54:11.078958,NaT,57 days 03:54:11.078958,NaT,NaT,NaT,NaT,2019-12-11,NaT,2020-02-05 23:00:00.000000,NaT,NaN,2019-12-11 09:40:33.000000,NaN,NaN,NaT,NaT,pending,2020-12-18 22:47:47.993817,2020-03-20 14:12:53.208246,2020-12-18 22:47:48.008850,NaT,2019-12,2019-12-11,0,0,14,19-Tarde,1,-1
4,7,0,NaN,191,1,2019-12-10 19:13:35.825460,NaT,100.0,NaN,rejected,NaN,regular,nice,NaT,NaT,29 days 23:59:59.999581,NaT,NaT,NaT,NaT,NaT,NaT,2020-01-09 19:13:35.825041,NaT,NaN,2019-12-11 09:46:59.777728,NaN,NaN,NaT,NaT,nice,2019-12-11 09:46:59.779773,NaT,NaT,NaT,2019-12,NaT,0,0,14,19-Tarde,1,-1
5,9,0,NaN,430,1,2019-12-10 19:14:41.668754,NaT,100.0,NaN,money_back,NaN,regular,nice,0 days 04:45:18.331246,NaT,29 days 03:45:18.331246,NaT,-125 days +11:54:30,NaT,NaT,2019-12-11,2020-05-12 11:05:30.000000,2020-01-08 23:00:00.000000,NaT,NaN,2019-12-11 09:50:51.000000,NaN,NaN,NaT,NaT,completed,2020-11-04 12:55:30.918436,2020-03-20 14:12:53.263164,2020-05-12 14:20:09.505802,NaT,2019-12,2019-12-11,0,1,14,19-Tarde,1,-1
6,10,0,NaN,761,1,2019-12-10 19:16:10.880172,NaT,99.0,NaN,rejected,NaN,regular,nice,NaT,NaT,29 days 23:59:59.999434,NaT,NaT,NaT,NaT,NaT,NaT,2020-01-09 19:16:10.879606,NaT,NaN,2019-12-18 14:26:18.128407,NaN,NaN,NaT,NaT,nice,2019-12-18 14:26:18.136163,NaT,NaT,NaT,2019-12,NaT,0,0,14,19-Tarde,1,-1
7,11,0,NaN,735,1,2019-12-10 19:18:35.805897,NaT,100.0,NaN,money_back,NaN,regular,nice,1 days 04:41:24.194103,NaT,20 days 03:41:24.194103,NaT,0 days 00:00:00,NaT,NaT,2019-12-12,2019-12-30 23:00:00.000000,2019-12-30 23:00:00.000000,NaT,NaN,2019-12-11 16:53:50.943562,NaN,NaN,NaT,NaT,nice,2020-11-04 12:55:36.256240,NaT,NaT,NaT,2019-12,2019-12-12,0,1,14,19-Tarde,1,-1
8,13,0,NaN,406,1,2019-12-10 19:18:59.309847,NaT,100.0,NaN,direct_debit_rejected,NaN,regular,nice,1 days 04:41:00.690153,NaT,57 days 03:41:00.690153,NaT,NaT,NaT,NaT,2019-12-12,NaT,2020-02-05 23:00:00.000000,NaT,NaN,2019-12-11 16:56:12.640514,NaN,NaN,NaT,NaT,pending,2020-11-04 12:55:22.528038,2020-03-20 14:12:53.253927,2020-03-31 16:05:16.480570,NaT,2019-12,2019-12-12,0,0,14,19-Tarde,1,-1
9,28,0,NaN,140,1,2019-12-10 19:22:09.572973,NaT,100.0,NaN,money_back,NaN,regular,nice,NaT,NaT,29 days 03:37:50.427027,NaT,0 days 00:00:00,NaT,NaT,NaT,2020-01-08 23:00:00.000000,2020-01-08 23:00:00.000000,NaT,NaN,2019-12-11 09:59:38.319560,NaN,NaN,NaT,NaT,nice,2020-11-04 12:55:30.858159,NaT,NaT,NaT,2019-12,NaT,0,1,14,19-Tarde,1,-1


### Formato de fechas

In [ ]:
df = df_jo.copy()
timeFormat ='D' #S
df['created_at'] = df['created_at'].dt.to_period(timeFormat) #'Min')
df['created_at_fe'] = df['created_at_fe'].dt.to_period(timeFormat) #'Min')
df['updated_at'] = df['updated_at'].dt.to_period(timeFormat) #'Min')
df['updated_at_fe'] = df['updated_at_fe'].dt.to_period(timeFormat) #'Min')

df['to_receive_ini'] = pd.to_timedelta(df['to_receive_ini']).round(timeFormat)
#df['to_receive_ini'] = df['to_receive_ini'].timedelta(seconds=math.ceil(df['to_receive_ini'].total_seconds()))

df['to_receive_bank'] = pd.to_timedelta(df['to_receive_bank']).round(timeFormat)
df['to_reimbur'] = pd.to_timedelta(df['to_reimbur']).round(timeFormat)
df['to_reimbur_cash'] = pd.to_timedelta(df['to_reimbur_cash']).round(timeFormat)
df['to_end'] = pd.to_timedelta(df['to_end']).round(timeFormat)
df['to_send'] = pd.to_timedelta(df['to_send']).round(timeFormat)

df['money_back_date'] = df['money_back_date'].dt.to_period(timeFormat)
df['send_at'] = df['send_at'].dt.to_period(timeFormat)
df['paid_at'] = df['paid_at'].dt.to_period(timeFormat)
df['moderated_at'] = df['moderated_at'].dt.to_period(timeFormat)
df['from_date'] = df['from_date'].dt.to_period(timeFormat)
df['to_date'] = df['to_date'].dt.to_period(timeFormat)

df['reco_creation'] = df['reco_creation'].dt.to_period(timeFormat)
df['reco_last_update'] = df['reco_last_update'].dt.to_period(timeFormat)
display(df.head(5))

### cohort_analysis_2

In [ ]:
cohort_analysis_2 = (
    df_jo.query('stat_fe == "accepted" | stat_cr == "money_back"')
        .groupby(['user_id', 'Mes_created_at'], as_index=False)
        .agg(
            total_paid_fees=('fee', lambda x: x[df_jo.loc[x.index, 'stat_fe'] == 'accepted'].sum()),            
            # Contar los valores únicos de 'stat_cr' donde su valor sea 'money_back'
            total_paid_cr=('amount', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].unique().sum()),
            # Contar los valores únicos de 'id_cr' donde 'stat_cr' es igual a 'money_back'
            Num_Solicitudes=('id_cr', lambda x: x[df_jo.loc[x.index, 'stat_cr'] == 'money_back'].nunique())
        )
)
# Restablecer el índice para un DataFrame limpio (opcional, ya garantizado por as_index=False)
cohort_analysis_2.reset_index(drop=True, inplace=True)

# Calcular el índice como porcentaje entre 'total_paid_fees' y 'total_paid_cr'
cohort_analysis_2['cash_index'] = (
    cohort_analysis_2['total_paid_fees'] / cohort_analysis_2['total_paid_cr'] ) * 100

# Reemplazar valores 'inf' con 0 para manejar divisiones por cero
cohort_analysis_2['cash_index'] = cohort_analysis_2['cash_index'].replace(np.inf, 0)

# Calcular la fecha del último pedido por usuario a partir del DataFrame original
df_jo['created_at'] = pd.to_datetime(df_jo['created_at'])  # Asegurarse de que el formato sea datetime
last_order_per_user = (
    df_jo.groupby('user_id')['created_at']
    .max()  # Obtener la fecha más reciente de pedido para cada usuario
    .dt.to_period('M')  # Convertir a periodo mensual
    .reset_index()  # Restablecer el índice para facilitar el merge
)

# Incorporar la fecha del último pedido en el DataFrame de análisis de cohortes
cohort_analysis_2 = pd.merge(
    cohort_analysis_2,
    last_order_per_user.rename(columns={'created_at': 'last_order'}),
    on='user_id',
    how='left'
)

display(cohort_analysis_2.head(5))

### Estudio de casos de clientes concretos (user_id)

* 102105
* 16391 
* 2002
* 13851
* 1987 
* ...
* 16391 # 2002, 1987, 13851, 16391, 102105

* user_ids = [13851] [2002] , 1987, 1946, 90, 526, 12934] #, 12274 54879 12441, 13851, 16391, 430,  63894,18730,10116,21465, 99000262]
* vips 12934 526
* 90 Este se esta gestionando mal: todos instant, con demoras y sin gestion por 
* 1946 Parece un ejemplo de buena gestion, al final tiene un instant y se le ha dado margen el las demoras.
* 1987 Parece un ejemplo de buen usuario, se pasa a instant para siempre.

Casos que esta examinando Alba:
* 19655
* 21465
* 14631


In [ ]:
display(cohort_analysis_2[cohort_analysis_2.user_id == 102105])

display(cohort_analysis_2[cohort_analysis_2.user_id == 16391])
display(cohort_analysis_2[cohort_analysis_2.user_id == 2002])
display(cohort_analysis_2[cohort_analysis_2.user_id == 13851])
display(cohort_analysis_2[cohort_analysis_2.user_id == 1987]) #.tail(60))

In [ ]:
fields = ['id_cr','created_at','transfer_type','type','stat_cr' ,'amount','fee','n_fees','n_backs', # 'good_user',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment' # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          #'to_receive_ini','to_receive_bank' #,'to_reimbur_cash', 'updated_at', 'to_send','send_at','moderated_at'
]

user_id = 2002# 16391 # 2002, 1987, 13851, 16391, 102105
display(cohort_analysis_2[cohort_analysis_2.user_id == user_id])

#print("Casos segun Cash Request ID")
pd.options.display.max_columns = None
for id in ([-8177]): # 16391 20108, 20104, 20112,
    df_t = df[df['id_cr'] == id].sort_values(['created_at','created_at_fe']).reset_index()
    print(f"Cash Request ID: {id}")
    display(df_t[fields])

user_ids = [user_id] 
pd.options.display.max_columns = None
#print("Casos segun Cash User ID")
for id in (user_ids):
    df_t = df[(df['user_id'] == id)]#.reset_index()
    df_t = df_t[df_t['stat_cr'] == 'money_back']
    df_t = df_t[df_t['stat_fe'] == 'accepted']
    
    df_t = df_t.sort_values(['created_at','created_at_fe']).reset_index(drop=True)
    #df_t.set_index('id_cr', inplace=True)
    print(f"Only money_back - user_id {id}")
    display(df_t[fields])
    df_t = df[(df['user_id'] == id) ].sort_values(['created_at','created_at_fe']).reset_index(drop=True)

    print(f"user_id {id}")
    display(df_t[fields])

### Estudio 2 de clientes

In [ ]:
#column_names = ["cr_id", "user_id"]
fields = ['id_cr','user_id','created_at','transfer_type','type','stat_cr' ,'amount','fee','n_fees','n_backs','good_user',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment' # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          #'to_receive_ini','to_receive_bank' #,'to_reimbur_cash', 'updated_at', 'to_send','send_at','moderated_at'
]
df = df.reindex(columns=fields)
df.reset_index(drop=True, inplace=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
#display(df.head(5))
display(df[df['user_id'] == 2002]) #.reset_index()

### Estudio de top FEES

In [ ]:
pd.options.display.max_rows = None
tops = df_jo[df_jo['stat_cr' ]== 'money_back'].groupby('user_id').agg(fees=('fee','sum'))
tops = df_jo[df_jo['stat_fe'] == 'accepted'  ].groupby('user_id').agg(fees=('fee','sum'))
#display(df_jo[tops])
top_users = tops.sort_values(by='fees', ascending=False).iloc[:10].reset_index()
display(top_users)
#top_users = tops.sort_values(by='fees', ascending=True).iloc[:10]
#display(top_users)

#display(df_jo[top_users])